# MoCo

[李沐讲论文](https://www.youtube.com/watch?v=pXvMXfPJZ2M&list=PLFXJ6jwg0qW-7UM8iUTj3qKqdhbQULP5I&index=11)

![](01.png)

## 对比学习 contrast learning

![](02.png)

1. InfoNCE的损失函数，分母是正样例，分母是负样例

![](03.png)

1. 将对比学习问题看作是一个动态字典查询问题。比如单体检测问题，从原始图片生成两个样本，一个是query，一个是$key^+$，其他样本都是负样本$key_i^-$。所有样本就是这个字典的key
   
2. 字典的key都要用相似编码器得到。因为在训练过程中，不断会有新的key进入字典，而每次新的key的加入，都是一次训练，都要更新编码器的参数。所以就导致字典中不同的key使用的编码器不一样。这就容易导致不一致。比如如果key1,key2用的不同的编码器，key1和query用的一样的，而实际key2才是positive的，但是由于用的编码器差别过大，导致选择的是key1

3. MoCo 的提出就是为了解决字典太大和一致性问题。在编码query和key的时候，刚开始用的是一样的编码参数。在进行gradient back propagation的时候，并不会回传到key的编码器。这样key就可以选取很多，解决了large的问题。在更新完query encoder参数之后，会把这个参数也更新到key的encoder，但是采用的是动量更新的方式，即$\theta_k^i = m \theta_k^{i-1} + (1-m)q^{i}$。这个m可以选取的很小，这样就保证key encoder的参数更新比较慢，新加入的key和之前的key基本使用相似的参数生成的。解决了consistent的问题。同时当queue size过大的时候，移除最早的key，也是参数最老的key

![](04.png)

1. pretext task：代理任务，其实就是在视觉领域，自动可以生成一些类似标签的东西方便后面作自监督训练。比如前面说的instance discriminate。或者把一张图片分成九宫格子，告诉你中间一张，再给你另外一个，判断是其他8个位置的哪一个（文中的 eight position）

2. 文中的损失函数，典型的NCE的loss。只不过有个K，本来有很多负样本，计算量太大。我们可以在所有负样本中抽取K个来计算

![](05.png)

1. 使用queue来储存之前的key，可以把mini-batch size跟我们训练的size分离开，解决了GPU内存的限制。同时每次移除的都是最老的key，保证consistent

2. end-to-end 方法，编码器解码器的参数都参与训练更新。问题就受限与字典大小。如果字典很大，每个mini-batch就要有很多样本

3. memory-bank 方法，提前把样本抽取，存储在memory bank中，每次训练，从其中采样，参与训练。训练好了之后，会更新key encoder的参数，然后用新的encoder更新memory bank中对应的key。它的问题就是一致性

![](06.png)